In [ ]:
BUCKET_NAME="beeflow-dev-metadata-dumps"

In [ ]:
import pandas as pd
import boto3

In [ ]:
s3 = boto3.client('s3') 

In [ ]:
def get_s3_csv(file: str):
    obj = s3.get_object(Bucket=BUCKET_NAME, Key=file)
    return pd.read_csv(obj['Body'])


In [ ]:
from enum import Enum

class System(Enum):
    MWAA = "mwaa"
    BEEFLOW = "serverless"
    
class Stats(Enum):
    DAG_RUNS = "dagrun"
    TASKS = "taskinstance"

In [ ]:
def get_stats(system: System, date: str, stats: Stats):
    return get_s3_csv(f"{system.value}/export/{stats.value}/dt={date}/{stats.value}.csv")

In [ ]:
def filter_on_dag_ids(df, dag_ids):
    return df[df['dag_id'].isin(dag_ids)]

In [ ]:
mwaa_shard_10_tasks = filter_on_dag_ids(get_stats(system=System.MWAA, date="2022-12-07_21-57", stats=Stats.TASKS), dag_ids=["beeflow_experiment2"])

In [ ]:
beeflow_shard_10_tasks = filter_on_dag_ids(get_stats(system=System.BEEFLOW, date="2022-12-07_21-50", stats=Stats.TASKS), dag_ids=["beeflow_experiment2"])

In [ ]:
import plotly.express as px

In [ ]:
def gantt_task(df, range_x=None):
    fig = px.timeline(df, x_start="start_date", x_end="end_date", color="run_id", y="task_id", range_x=range_x)
    fig.update_yaxes(autorange="reversed")
    fig.show()

In [ ]:
shard_10_tasks_x_range = ("2022-12-07 20:40:00.000000+00:00", "2022-12-07 21:55:00.000000+00:00")

In [ ]:
gantt_task(mwaa_shard_10_tasks, range_x=shard_10_tasks_x_range)

In [ ]:
gantt_task(beeflow_shard_10_tasks, range_x=shard_10_tasks_x_range)

In [ ]:
mwaa_shard_10_dag_runs = filter_on_dag_ids(get_stats(system=System.MWAA, date="2022-12-07_21-57", stats=Stats.DAG_RUNS), dag_ids=["beeflow_experiment2"])

In [ ]:
beeflow_shard_10_dag_runs = filter_on_dag_ids(get_stats(system=System.BEEFLOW, date="2022-12-07_21-50", stats=Stats.DAG_RUNS), dag_ids=["beeflow_experiment2"])

In [ ]:
def gantt_dag_runs(df, **kwargs):
    fig = px.timeline(df, x_start="start_date", x_end="end_date", color="run_id", **kwargs)
    fig.update_yaxes(autorange="reversed")
    fig.show()

In [ ]:
gantt_dag_runs(mwaa_shard_10_dag_runs, range_x=shard_10_tasks_x_range)

In [ ]:
gantt_dag_runs(beeflow_shard_10_dag_runs, range_x=shard_10_tasks_x_range)

In [ ]:
def average_dag_run_duration(df):
    start_ms_time = pd.to_datetime(df['start_date'])
    end_ms_time = pd.to_datetime(df['end_date'])
    df['duration'] = end_ms_time.sub(start_ms_time).dt.total_seconds()
    return df['duration'].mean()

In [ ]:
average_dag_run_duration(mwaa_shard_10_dag_runs)

In [ ]:
average_dag_run_duration(beeflow_shard_10_dag_runs)